# Agregando dados com pandas e numpy

## Sobre os dados
Neste notebook, trabalharemos com dois conjuntos de dados:
- Preço das ações do Facebook ao longo de 2018 (obtido usando o pacote [`stock_analysis`](https://github.com/stefmolin/stock-analysis)).
- Dados meteorológicos diários de NYC da API dos [Centros Nacionais de Informações Ambientais (NCEI)](https://www.ncdc.noaa.gov/cdo-web/webservices/v2).

*Nota: O NCEI faz parte da Administração Nacional Oceânica e Atmosférica (NOAA) e, como você pode ver na URL da API, este recurso foi criado quando o NCEI era chamado de NCDC. Caso a URL deste recurso mude no futuro, você pode procurar por "API de clima NCEI" para encontrar a versão atualizada.*

## Informações sobre os dados meteorológicos

Significados dos dados:
- `AWND`: velocidade média do vento
- `PRCP`: precipitação em milímetros
- `SNOW`: queda de neve em milímetros
- `SNWD`: profundidade da neve em milímetros
- `TMAX`: temperatura máxima diária em Celsius
- `TMIN`: temperatura mínima diária em Celsius

## Configuração

In [1]:
import numpy as np
import pandas as pd

fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True).assign(
    trading_volume=lambda x: pd.cut(x.volume, bins=3, labels=['low', 'med', 'high'])
)
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903,low
2018-01-03,181.88,184.78,181.3300,184.67,16886563,low
2018-01-04,184.90,186.21,184.0996,184.33,13880896,low
2018-01-05,185.59,186.90,184.9300,186.85,13574535,low
2018-01-08,187.20,188.90,186.3300,188.28,17994726,low


In [2]:
weather = pd.read_csv('data/weather_by_station.csv', index_col='date', parse_dates=True)
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.0,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"


Antes de começarmos qualquer cálculo, vamos garantir que o `pandas` não exibirá os números em notação científica. Iremos modificar o formato de exibição dos números float. O formato que aplicaremos é `.2f`, que exibirá os números float com 2 dígitos após o ponto decimal:

In [3]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Resumindo DataFrames
Aprendemos sobre `agg()` no notebook [`2-dataframe_operations.ipynb`](./2-dataframe_operations.ipynb) quando aprendemos sobre cálculos de janela; no entanto, podemos chamar isso diretamente no dataframe para agregar seu conteúdo em uma única série:

In [6]:
fb.head(2)

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.55,181.42,18151903,low
2018-01-03,181.88,184.78,181.33,184.67,16886563,low


In [5]:
fb.agg({
    'open': 'mean', 
    'high': 'max', 
    'low': 'min', 
    'close': 'mean', 
    'volume': 'sum'
})

open            171.45
high            218.62
low             123.02
close           171.51
volume   6949682394.00
dtype: float64

Podemos usar isso para encontrar o total de queda de neve e precipitação registrados no Central Park em 2018:

In [7]:
weather.query('station == "GHCND:USW00094728"')\
    .pivot(columns='datatype', values='value')[['SNOW', 'PRCP']]\
    .sum()

datatype
SNOW   1007.00
PRCP   1665.30
dtype: float64

Isso é equivalente a passar `'sum'` para `agg()`:

In [8]:
weather.query('station == "GHCND:USW00094728"')\
    .pivot(columns='datatype', values='value')[['SNOW', 'PRCP']]\
    .agg('sum')

datatype
SNOW   1007.00
PRCP   1665.30
dtype: float64

Observe que não estamos limitados a fornecer uma única agregação por coluna. Podemos passar uma lista, e obteremos um dataframe em vez de uma série. Valores nulos são colocados onde não temos um resultado de cálculo para exibir:

In [9]:
fb.agg({
    'open': 'mean',
    'high': ['min', 'max'],
    'low': ['min', 'max'],
    'close': 'mean'
})

,open,high,low,close
mean,171.45,NaN,NaN,171.51
min,NaN,129.74,123.02,NaN
max,NaN,218.62,214.27,NaN


## Usando `groupby()`
Frequentemente, não queremos agregar em todo o dataframe, mas sim em grupos dentro dele. Para isso, podemos usar `groupby()` antes da agregação. Se agruparmos pela coluna `trading_volume`, obteremos uma linha para cada um dos valores que ela assume:

In [12]:
fb.groupby('trading_volume', observed=False).mean()

,open,high,low,close,volume
trading_volume,,,,,
low,171.36,173.46,169.31,171.43,24547207.71
med,175.82,179.42,172.11,175.14,79072559.12
high,167.73,170.48,161.57,168.16,141924023.33


Depois de chamarmos `groupby()`, ainda podemos selecionar colunas para agregação:

In [14]:
fb.groupby('trading_volume', observed=False)['close'].agg(['min', 'max', 'mean'])

,min,max,mean
trading_volume,,,
low,124.06,214.67,171.43
med,152.22,217.50,175.14
high,160.06,176.26,168.16


Ainda podemos fornecer um dicionário especificando as agregações a serem realizadas, mas passar uma lista para uma coluna resultará em um índice hierárquico para as colunas:

In [16]:
fb_agg = fb.groupby('trading_volume', observed=False).agg({
    'open': 'mean',
    'high': ['min', 'max'],
    'low': ['min', 'max'],
    'close': 'mean'
})
fb_agg

open   high           low         close
                 mean    min    max    min    max   mean
trading_volume                                          
low            171.36 129.74 216.20 123.02 212.60 171.43
med            175.82 162.85 218.62 150.75 214.27 175.14
high           167.73 161.10 180.13 149.02 173.75 168.16

O índice hierárquico nas colunas fica assim:

In [17]:
fb_agg.columns

MultiIndex([( 'open', 'mean'),
            ( 'high',  'min'),
            ( 'high',  'max'),
            (  'low',  'min'),
            (  'low',  'max'),
            ('close', 'mean')],
           )

Usando uma list comprehension, podemos unir os níveis (em uma tupla) com `_` em cada iteração:

In [18]:
fb_agg.columns = ['_'.join(col_agg) for col_agg in fb_agg.columns]
fb_agg.head()

,open_mean,high_min,high_max,low_min,low_max,close_mean
trading_volume,,,,,,
low,171.36,129.74,216.20,123.02,212.60,171.43
med,175.82,162.85,218.62,150.75,214.27,175.14
high,167.73,161.10,180.13,149.02,173.75,168.16


Também podemos criar um objeto `Grouper`, que pode agrupar os datetimes no índice. Aqui, encontramos o total trimestral de precipitação por estação:

In [30]:
weather.query('datatype == "PRCP"').groupby(
    ['station_name', pd.Grouper(freq='QE')]
)['value'].sum().unstack().sample(5, random_state=1)

date,2018-03-31,2018-06-30,2018-09-30,2018-12-31
station_name,,,,
"WANTAGH 1.1 NNE, NY US",279.90,216.80,472.50,277.20
"STATEN ISLAND 1.4 SE, NY US",379.40,295.30,438.80,409.90
"SYOSSET 2.0 SSW, NY US",323.50,263.30,355.50,459.90
"STAMFORD 4.2 S, CT US",338.00,272.10,424.70,390.00
"WAYNE TWP 0.8 SSW, NJ US",246.20,295.30,620.90,422.00


Note que podemos usar `filter()` para excluir alguns grupos da agregação. Aqui, mantemos apenas os grupos cujos nomes terminam em "NY US" no atributo `name` do grupo, que neste caso é o nome da estação:

In [38]:
weather.groupby('station_name').filter( # station names with "NY US" in them
    lambda x: x.name.endswith('NY US')
).query('datatype == "SNOW"').groupby('station_name')['value'].sum()

station_name
ALBERTSON 0.2 SSE, NY US           1087.00
AMITYVILLE 0.1 WSW, NY US           434.00
AMITYVILLE 0.6 NNE, NY US          1072.00
ARMONK 0.3 SE, NY US               1504.00
BROOKLYN 3.1 NW, NY US              305.00
CENTERPORT 0.9 SW, NY US            799.00
CENTERPORT, NY US                  1333.00
ELMSFORD 0.8 SSW, NY US             863.00
FLORAL PARK 0.4 W, NY US           1015.00
HICKSVILLE 1.3 ENE, NY US           716.00
JACKSON HEIGHTS 0.3 WSW, NY US      107.00
JFK INTERNATIONAL AIRPORT, NY US    855.00
LA GUARDIA AIRPORT, NY US           892.00
LOCUST VALLEY 0.3 E, NY US            0.00
LYNBROOK 0.3 NW, NY US              325.00
MASSAPEQUA 0.9 SSW, NY US            41.00
MIDDLE VILLAGE 0.5 SW, NY US       1249.00
NEW HYDE PARK 1.6 NE, NY US           0.00
NEW YORK 8.8 N, NY US                 0.00
NORTH WANTAGH 0.4 WSW, NY US        471.00
NY CITY CENTRAL PARK, NY US        1007.00
PLAINEDGE 0.4 WSW, NY US            610.00
PLAINVIEW 0.4 ENE, NY US           1360.0

Vamos ver quais meses têm mais precipitação. Primeiro, precisamos agrupar por dia e calcular a média da precipitação entre as estações. Depois, podemos agrupar por mês e somar a precipitação resultante. Usaremos `nlargest()` para obter os 5 meses com mais precipitação:

In [52]:
weather.query('datatype == "PRCP"')\
    .groupby(level=0)['value'].mean()\
    .groupby(pd.Grouper(freq='ME')).sum().nlargest()

date
2018-11-30   210.59
2018-09-30   193.09
2018-08-31   192.45
2018-07-31   160.98
2018-02-28   158.11
Name: value, dtype: float64

## Tabelas dinâmicas e crosstabs

Vimos tabelas dinâmicas em [`ch_03/4-reshaping_data.ipynb`](../ch_03/4-reshaping_data.ipynb); no entanto, não conseguimos fornecer nenhuma agregação. Com `pivot_table()`, por padrão, obtemos a média. Em sua forma mais simples, fornecemos uma coluna para colocar ao longo das colunas:

In [65]:
fb.head(2)

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.55,181.42,18151903,low
2018-01-03,181.88,184.78,181.33,184.67,16886563,low


In [70]:
fb.pivot_table(columns='trading_volume', observed=False)

trading_volume,low,med,high
close,171.43,175.14,168.16
high,173.46,179.42,170.48
low,169.31,172.11,161.57
open,171.36,175.82,167.73
volume,24547207.71,79072559.12,141924023.33


Ao colocar o volume de negociação no índice, obtemos a transposição:

In [67]:
fb.pivot_table(index='trading_volume', observed=False)

,close,high,low,open,volume
trading_volume,,,,,
low,171.43,173.46,169.31,171.36,24547207.71
med,175.14,179.42,172.11,175.82,79072559.12
high,168.16,170.48,161.57,167.73,141924023.33


Com `pivot()`, não conseguimos lidar com índices de vários níveis ou índices com valores repetidos. Por esse motivo, não conseguimos colocar os dados meteorológicos no formato amplo. O método `pivot_table()` resolve esse problema:

In [73]:
weather.reset_index()

,date,datatype,station,value,station_name
0,2018-01-01,PRCP,GHCND:US1CTFR0039,0.00,"STAMFORD 4.2 S, CT US"
1,2018-01-01,PRCP,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2,2018-01-01,SNOW,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
3,2018-01-01,PRCP,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"
4,2018-01-01,SNOW,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"
...,...,...,...,...,...
80251,2018-12-31,WDF5,GHCND:USW00094789,130.00,"JFK INTERNATIONAL AIRPORT, NY US"
80252,2018-12-31,WSF2,GHCND:USW00094789,9.80,"JFK INTERNATIONAL AIRPORT, NY US"
80253,2018-12-31,WSF5,GHCND:USW00094789,12.50,"JFK INTERNATIONAL AIRPORT, NY US"
80254,2018-12-31,WT01,GHCND:USW00094789,1.00,"JFK INTERNATIONAL AIRPORT, NY US"


In [80]:
weather.reset_index().pivot_table(
    index=['date', 'station', 'station_name'], 
    columns='datatype', 
    values='value',
    aggfunc='median'
).reset_index().tail()

datatype,date,station,station_name,AWND,DAPR,MDPR,PGTM,PRCP,SNOW,SNWD,...,WSF5,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
28740,2018-12-31,GHCND:USW00054787,"FARMINGDALE REPUBLIC AIRPORT, NY US",5.00,NaN,NaN,2052.00,28.70,NaN,NaN,...,15.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28741,2018-12-31,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",NaN,NaN,NaN,NaN,25.90,0.00,0.00,...,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28742,2018-12-31,GHCND:USW00094741,"TETERBORO AIRPORT, NJ US",1.70,NaN,NaN,1954.00,29.20,NaN,NaN,...,8.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28743,2018-12-31,GHCND:USW00094745,"WESTCHESTER CO AIRPORT, NY US",2.70,NaN,NaN,2212.00,24.40,NaN,NaN,...,11.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28744,2018-12-31,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",4.10,NaN,NaN,NaN,31.20,0.00,0.00,...,12.50,1.00,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Podemos usar a função `pd.crosstab()` para criar uma tabela de frequência. Por exemplo, se quisermos ver quantos dias de negociação de baixo, médio e alto volume o estoque do Facebook teve cada mês, podemos usar crosstab:

In [81]:
fb.head(2)

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.55,181.42,18151903,low
2018-01-03,181.88,184.78,181.33,184.67,16886563,low


In [83]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month']
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,20,19,15,20,22,21,18,23,19,23,21,19
med,1,0,4,1,0,0,2,0,0,0,0,0
high,0,0,2,0,0,0,1,0,0,0,0,0


Podemos normalizar com os totais das linhas ou das colunas usando o parâmetro `normalize`. Isso mostra a porcentagem do total:

In [84]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'],
    normalize='columns'
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,0.95,1.00,0.71,0.95,1.00,1.00,0.86,1.00,1.00,1.00,1.00,1.00
med,0.05,0.00,0.19,0.05,0.00,0.00,0.10,0.00,0.00,0.00,0.00,0.00
high,0.00,0.00,0.10,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00


Se quisermos realizar um cálculo diferente de contar a frequência, podemos passar a coluna na qual executar o cálculo para `values` e a função a ser usada para `aggfunc`:

In [86]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'],
    values=fb.close,
    aggfunc='mean'
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,185.24,180.27,177.07,163.29,182.93,195.27,201.92,177.49,164.38,154.19,141.64,137.16
med,179.37,NaN,164.76,174.16,NaN,NaN,194.28,NaN,NaN,NaN,NaN,NaN
high,NaN,NaN,164.11,NaN,NaN,NaN,176.26,NaN,NaN,NaN,NaN,NaN


Também podemos obter subtotais de linhas e colunas com o parâmetro `margins`. Vamos contar quantas vezes cada estação registrou neve por mês e incluir os subtotais:

In [87]:
weather.head(2)

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.00,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"


In [90]:
snow_data = weather.query('datatype == "SNOW"')
pd.crosstab(
    index=snow_data.station_name,
    columns=snow_data.index.month,
    colnames=['month'],
    values=snow_data.value,
    aggfunc=lambda x: (x > 0).sum(),
    margins=True, # show row and column subtotals
    margins_name='total observations of snow' # name the subtotals
)

month,1,2,3,4,5,6,7,8,9,10,11,12,total observations of snow
station_name,,,,,,,,,,,,,
"ALBERTSON 0.2 SSE, NY US",3.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,9
"AMITYVILLE 0.1 WSW, NY US",1.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3
"AMITYVILLE 0.6 NNE, NY US",3.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8
"ARMONK 0.3 SE, NY US",6.00,4.00,6.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,3.00,23
"BLOOMINGDALE 0.7 SSE, NJ US",2.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"WESTFIELD 0.6 NE, NJ US",3.00,0.00,4.00,1.00,0.00,NaN,0.00,0.00,0.00,NaN,1.00,NaN,9
"WOODBRIDGE TWP 1.1 ESE, NJ US",4.00,1.00,3.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,11
"WOODBRIDGE TWP 1.1 NNE, NJ US",2.00,1.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,7


<hr>
<div>
    <a href="./2-dataframe_operations.ipynb">
        <button>&#8592; Previous Notebook</button>
    </a>
    <a href="./4-time_series.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>